In [1]:
seed = 0
import torch
torch.manual_seed(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
torch.use_deterministic_algorithms(True)

In [2]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
from utils_LS import train, test, edgeindex2adj, adj2edgeindex, laplacian, non_smooth_label_metric
from models import GCN, GAT, LP
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
pubmed = Planetoid(root='.', name='Pubmed')

In [3]:
# dataset = citeseer
# model = GCN(dataset.num_features, 16, dataset.num_classes)

dataset = cora
# model = GCN(dataset.num_features, 16, dataset.num_classes)

# dataset = pubmed
# model = GCN(dataset.num_features, 16, dataset.num_classes)
# 
# dataset = citeseer
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

# dataset = cora
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

# dataset = pubmed
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

data = dataset[0]

In [4]:
data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum()

(tensor(140), tensor(500), tensor(1000))

In [5]:
k = 5
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False
print(data.train_mask.sum())

tensor(35)


In [6]:
adj = edgeindex2adj(data.edge_index, data.x.shape[0])
lap = laplacian(adj, typ='normalized')
lap.isnan().any()

tensor(False)

In [7]:
total_test_acc = []
avg_val_acc = 0
for seed in [42, 1234, 2021]:
    torch.manual_seed(seed)
    model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=50, min_lr=0.01)
    best_val_acc = final_test_acc = 0
    for epoch in range(1, 200):
        loss = train(model, data, optimizer, lap, scheduler=None, loss='smooth_label', alpha=0.00001)
        train_acc, val_acc, tmp_test_acc = test(model, data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')
    total_test_acc.append(test_acc)
    avg_val_acc += best_val_acc
print(f'Average Val Acc: {avg_val_acc/3:.4f}')
print(f'Average Test Acc: {np.mean(total_test_acc):.4f} ± {np.std(total_test_acc):.4f}')
torch.save(model.state_dict(), f'./models_LS/{dataset.name}_GAT_{k}.pt')
preds = model(data.x, data.edge_index).argmax(dim=-1)
nsm = non_smooth_label_metric(dataset, preds)
nsm

Epoch: 001, Loss: 1.9686776399612427, Train: 0.5143, Val: 0.2860, Test: 0.3030
Epoch: 002, Loss: 1.8849784135818481, Train: 0.7143, Val: 0.3680, Test: 0.3740
Epoch: 003, Loss: 1.7385286092758179, Train: 0.7429, Val: 0.4080, Test: 0.4030
Epoch: 004, Loss: 1.5868669748306274, Train: 0.8857, Val: 0.4420, Test: 0.4270
Epoch: 005, Loss: 1.4758074283599854, Train: 0.9714, Val: 0.4780, Test: 0.4720
Epoch: 006, Loss: 1.2254014015197754, Train: 1.0000, Val: 0.5180, Test: 0.5020
Epoch: 007, Loss: 1.129117727279663, Train: 1.0000, Val: 0.5640, Test: 0.5490
Epoch: 008, Loss: 1.0419087409973145, Train: 1.0000, Val: 0.6140, Test: 0.6020
Epoch: 009, Loss: 1.0858254432678223, Train: 1.0000, Val: 0.6420, Test: 0.6390
Epoch: 010, Loss: 0.8004163503646851, Train: 1.0000, Val: 0.6640, Test: 0.6630
Epoch: 011, Loss: 0.6679726243019104, Train: 1.0000, Val: 0.6680, Test: 0.6770
Epoch: 012, Loss: 0.6991634964942932, Train: 1.0000, Val: 0.6760, Test: 0.6900
Epoch: 013, Loss: 0.9292731881141663, Train: 1.0000, 

0.1

tensor(30)